In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_column',False)

In [3]:
match = pd.read_csv(r'../Data/IPL_Matches_2008_2022.csv')
delivery = pd.read_csv(r'../Data/IPL_Ball_by_Ball_2008_2022.csv')

In [4]:
delivery.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [5]:
delivery.groupby(['ID','innings'])['total_run'].sum()

ID       innings
335982   1          222
         2           82
335983   1          240
         2          207
335984   1          129
                   ... 
1312198  2          193
1312199  1          157
         2          161
1312200  1          130
         2          133
Name: total_run, Length: 1928, dtype: int64

In [6]:
new = delivery.merge(match,on='ID',how='inner')

In [7]:
new.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon


In [8]:
new = new[new.innings == 1]

In [9]:
target_df = new.groupby(['ID'])['total_run'].sum().reset_index()

In [10]:
new = new.merge(target_df,on='ID')

In [11]:
new.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run_x,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,total_run_y
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,130
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,130
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,130
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,130
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,130


In [12]:
new['isWicketDelivery'].unique()

array([0, 1], dtype=int64)

In [13]:
final = new.groupby(['ID','overs','BattingTeam','Team1','Team2','Venue','total_run_y',]).agg({'total_run_x':'sum','isWicketDelivery':'sum'}).reset_index()

In [14]:
final['overs'] = final['overs'] + 1

In [15]:
final.head(20)

,ID,overs,BattingTeam,Team1,Team2,Venue,total_run_y,total_run_x,isWicketDelivery
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,3,0
1,335982,2,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,18,0
2,335982,3,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,6,0
3,335982,4,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,23,0
4,335982,5,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,10,0
5,335982,6,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,1,1
6,335982,7,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,7,0
7,335982,8,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,5,0
8,335982,9,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,4,0
9,335982,10,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,10,0


In [16]:
final['CurrentScore'] = final.groupby(['ID'])['total_run_x'].cumsum()
final['Wicket'] = final.groupby(['ID'])['isWicketDelivery'].cumsum()

In [17]:
final['BowlingTeam'] = np.where(final.BattingTeam == final.Team1 , final.Team2, final.Team1)

In [19]:
final[['BattingTeam','BowlingTeam','Venue','overs','CurrentScore','Wicket','total_run_y']]

,BattingTeam,BowlingTeam,Venue,overs,CurrentScore,Wicket,total_run_y
0,Kolkata Knight Riders,Royal Challengers Bangalore,M Chinnaswamy Stadium,1,3,0,222
1,Kolkata Knight Riders,Royal Challengers Bangalore,M Chinnaswamy Stadium,2,21,0,222
2,Kolkata Knight Riders,Royal Challengers Bangalore,M Chinnaswamy Stadium,3,27,0,222
3,Kolkata Knight Riders,Royal Challengers Bangalore,M Chinnaswamy Stadium,4,50,0,222
4,Kolkata Knight Riders,Royal Challengers Bangalore,M Chinnaswamy Stadium,5,60,0,222
...,...,...,...,...,...,...,...
18817,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",16,98,6,130
18818,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",17,104,6,130
18819,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",18,120,7,130
18820,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",19,123,7,130


In [21]:
final.head(40)

,ID,overs,BattingTeam,Team1,Team2,Venue,total_run_y,total_run_x,isWicketDelivery,CurrentScore,Wicket,BowlingTeam
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,3,0,3,0,Royal Challengers Bangalore
1,335982,2,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,18,0,21,0,Royal Challengers Bangalore
2,335982,3,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,6,0,27,0,Royal Challengers Bangalore
3,335982,4,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,23,0,50,0,Royal Challengers Bangalore
4,335982,5,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,10,0,60,0,Royal Challengers Bangalore
5,335982,6,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,1,1,61,1,Royal Challengers Bangalore
6,335982,7,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,7,0,68,1,Royal Challengers Bangalore
7,335982,8,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,5,0,73,1,Royal Challengers Bangalore
8,335982,9,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,4,0,77,1,Royal Challengers Bangalore
9,335982,10,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,10,0,87,1,Royal Challengers Bangalore


In [24]:
final = final[~(final.overs == 20)]

In [26]:
final.head()

,ID,overs,BattingTeam,Team1,Team2,Venue,total_run_y,total_run_x,isWicketDelivery,CurrentScore,Wicket,BowlingTeam
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,3,0,3,0,Royal Challengers Bangalore
1,335982,2,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,18,0,21,0,Royal Challengers Bangalore
2,335982,3,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,6,0,27,0,Royal Challengers Bangalore
3,335982,4,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,23,0,50,0,Royal Challengers Bangalore
4,335982,5,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,222,10,0,60,0,Royal Challengers Bangalore


In [29]:
final = final[['BattingTeam','BowlingTeam','Venue','overs','CurrentScore','Wicket','total_run_y']]

In [30]:
final.columns = ['BattingTeam','BowlingTeam','Venue','Overs','CurrentScore','Wicket','Target']

In [31]:
final.to_csv('final.csv', index=False)